In [69]:
import requests
import pandas as pd
import ast
import os
import time
import numpy as np

In [15]:
stock = 'VNM'
auth_token ='Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSIsImtpZCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSJ9.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4iLCJhdWQiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4vcmVzb3VyY2VzIiwiZXhwIjoxODg5NjIyNTMwLCJuYmYiOjE1ODk2MjI1MzAsImNsaWVudF9pZCI6ImZpcmVhbnQudHJhZGVzdGF0aW9uIiwic2NvcGUiOlsiYWNhZGVteS1yZWFkIiwiYWNhZGVteS13cml0ZSIsImFjY291bnRzLXJlYWQiLCJhY2NvdW50cy13cml0ZSIsImJsb2ctcmVhZCIsImNvbXBhbmllcy1yZWFkIiwiZmluYW5jZS1yZWFkIiwiaW5kaXZpZHVhbHMtcmVhZCIsImludmVzdG9wZWRpYS1yZWFkIiwib3JkZXJzLXJlYWQiLCJvcmRlcnMtd3JpdGUiLCJwb3N0cy1yZWFkIiwicG9zdHMtd3JpdGUiLCJzZWFyY2giLCJzeW1ib2xzLXJlYWQiLCJ1c2VyLWRhdGEtcmVhZCIsInVzZXItZGF0YS13cml0ZSIsInVzZXJzLXJlYWQiXSwianRpIjoiMjYxYTZhYWQ2MTQ5Njk1ZmJiYzcwODM5MjM0Njc1NWQifQ.dA5-HVzWv-BRfEiAd24uNBiBxASO-PAyWeWESovZm_hj4aXMAZA1-bWNZeXt88dqogo18AwpDQ-h6gefLPdZSFrG5umC1dVWaeYvUnGm62g4XS29fj6p01dhKNNqrsu5KrhnhdnKYVv9VdmbmqDfWR8wDgglk5cJFqalzq6dJWJInFQEPmUs9BW_Zs8tQDn-i5r4tYq2U8vCdqptXoM7YgPllXaPVDeccC9QNu2Xlp9WUvoROzoQXg25lFub1IYkTrM66gJ6t9fJRZToewCt495WNEOQFa_rwLCZ1QwzvL0iYkONHS_jZ0BOhBCdW9dWSawD6iF1SIQaFROvMDH1rg'
url = f'https://restv2.fireant.vn/symbols/{stock}/full-financial-reports?type=1&year=2025&quarter=0&limit=6'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36'


params = {
    'type': '1',
    'year': '2025',
    'quarter': '0',
    'limit': '6'
}

headers = {
    'User-Agent': user_agent,
    'Authorization': auth_token
}

response = requests.get(url, headers=headers, params=params)
response = response.json()

data = pd.DataFrame(response)

#xuất excel dữ liệu thô
filename_raw = f'{stock}_financial_reports_raw.xlsx'
data.to_excel(filename_raw, index=False)


In [71]:
df_raw = pd.read_excel(filename_raw)

#đặt tên cho file đã qua xử lý
filename_processed = f'{stock}_financial_reports_processed.xlsx'

# đổi từ string sang python_language
df_raw['values'] = df_raw['values'].apply(ast.literal_eval)
#explode cột values thành nhiều hàng
df_raw = df_raw.explode('values')
df_normalized = pd.json_normalize(df_raw['values'])


#nối df_raw với df_normalized
df_raw = pd.concat([df_raw['name'].reset_index(drop=True), df_normalized], axis=1)
df_raw = df_raw[['name','year','value']]

# đánh số 1 số tên
danh_sach =  ['- Giá trị hao mòn lũy kế','- Nguyên giá']
mask = df_raw['name'].isin(danh_sach)
cumcount_series = df_raw.groupby('name').cumcount().astype(str)
df_raw['name'] =np.where(mask, df_raw['name'] + '_' + cumcount_series, df_raw['name'])

df_raw = df_raw.pivot_table(index='name', columns='year', values='value').reset_index()

# thứ tự đúng

#lưu file
df_raw.to_excel(filename_processed, index=False)

In [ ]:
import pandas as pd

# --- GIẢ LẬP DỮ LIỆU CỦA BẠN ---
# Giả sử đây là DataFrame đã được pivot xong, có các hàng đang lộn xộn
data = {
    'Chỉ tiêu': ['Lợi nhuận gộp', 'Doanh thu thuần', 'Giá vốn hàng bán'],
    '2024': [50, 100, 50],
    '2023': [45, 90, 45]
}
df_processed = pd.DataFrame(data)

print("--- BẢNG GỐC (LỘN XỘN) ---")
print(df_processed)

# ===================================================================
# ===== BẮT ĐẦU KẾ HOẠCH 15 PHÚT CỦA BẠN TỪ ĐÂY ===================
# ===================================================================

# 1 & 2. COPY-PASTE TỪ WEB VÀO MỘT CHUỖI NHIỀU DÒNG
# (Đây là phần bạn sẽ copy từ Fireant)
fireant_order_string = """
Doanh thu thuần
Giá vốn hàng bán
Lợi nhuận gộp
"""

# 3. DÙNG .splitlines() ĐỂ BIẾN CHUỖI THÀNH LIST
# .strip() để xóa các dòng trống ở đầu và cuối
thu_tu_dung = fireant_order_string.strip().splitlines()

print("\n--- Danh sách thứ tự chuẩn đã lấy được ---")
print(thu_tu_dung)

# 4. DÙNG LIST ĐÓ ĐỂ .reindex() LẠI DATAFRAME
print("\n--- Bắt đầu sắp xếp lại... ---")
df_final_sorted = df_processed.set_index('Chỉ tiêu').reindex(thu_tu_dung).reset_index()


# 5. CẢM NHẬN SỰ CHIẾN THẮNG
print("\n--- KẾT QUẢ CUỐI CÙNG (ĐÃ SẮP XẾP) ---")
print(df_final_sorted)